## use following code to import this class
```
import import_ipynb
from imgAugmentation import ImageProcess
```

In [ ]:
import os
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
from keras import models
from keras import layers
from keras import regularizers
from keras.applications import VGG16
from PIL import Image
import numpy as np

standard_width = 200
standard_height = 280

In [ ]:
class Predictor:
    
    def __init__(self):
        """
        Constructor of the Predictor with the model_weight trained beforehand
        """

        # initialize the pretrained self.model
        conv_base = VGG16(weights='imagenet',
                          include_top=False,
                          input_shape=(standard_height, standard_width, 3))

        # initilize the self.model
        self.model = models.Sequential()
        self.model.add(conv_base)
        self.model.add(layers.Flatten())
        self.model.add(layers.Dense(256, activation='relu'))
        self.model.add(layers.Dense(1,activation='sigmoid'))

        # compile the self.model
        self.model.compile(optimizer='Adam',
                      loss='binary_crossentropy',
                      metrics=['accuracy'])

        # load weights
        model_path = './model_save/'
        model_name = 'vgg16_solveOverlap.h5'
        if not os.path.exists(model_path):
            os.mkdirs(model_path)
        if os.path.isfile(model_path + model_name):
            self.model.load_weights(model_path + model_name)

    def predict_imgPath(self, path):
        """
        Given a path, the function return its prediction
        :param path: the path of the image, eg: ./data/poster_negative/n0.png"
        :return: The prediction of the image. The closer the return is to 1, the better the image predicted is, vice versa.
        """
        img = Image.open(path).convert('RGB')
        im = np.array(img) / 255.0
        self.predict(im)
        
    def predict_PIL(self, img):
        """
        Given an PIL format picture, the function return its prediction
        :param img: img is PIL format
        :return: The prediction of the image. The closer the return is to 1, the better the image predicted is, vice versa.
        """
        im = np.array(img.convert('RGB')) / 255.0
        self.predict(im)
    
    
    def predict(self, im):
        """
        Given a pic array, the function gives the predictions of each one in this array
        :param img: img is np.array() format
        :return: The final result of predictions
        """
        test_set = []
        test_set.append(im)
        assert len(np.array(test_set).shape) == 4
        results = self.model.predict(np.array(test_set))
        print(results)
        

In [ ]:
if __name__ == "__main__":
    
    # construct a predictor
    predictor = Predictor()
    
    # test path
    img_path = "./data/poster_negative/n1667.png"
    predictor.predict_imgPath(img_path)
    
    # test PIL
    img_path = "./data/poster_negative_overlap/drink/15_100010_0.png"
    img = Image.open(img_path)
    predictor.predict_PIL(img)